In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from tensorflow.keras import layers
from tensorflow.keras import losses

# requires update to tensorflow 2.4
# >>> conda activate PIC16B
# >>> pip install tensorflow==2.4
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
import sqlite3

def retrieve(word):
    '''
    queries recipe database for titles including a specified word
    Parameters:
        word (str): keyword for parsing the recipe data
    Returns:
        pandas DataFrame with relevant recipes
    '''
    
    with sqlite3.connect("recipes1M.db") as conn:
        cmd = \
        f"""
        SELECT R.title
        FROM recipes R
        WHERE R.title LIKE "%{word}%"
        """
        df = pd.read_sql_query(cmd, conn)
    
    return df

In [3]:
df = retrieve("salmon")

In [4]:
df

,title
0,Salmon & Salad a La SPORTZ
1,Curried Pumpkin and Smoked Salmon Soup
2,Grilled Rosemary Salmon Spedini
3,Garlic and Dill Salmon
4,Spicy Grilled Orange Salmon
...,...
10788,Creamy Smoked Salmon and Dill Frittata
10789,Salmon and Spaghetti Casserole
10790,Salmon Pot Pie
10791,Pesto-Crusted Salmon Fillet With Citrus-Soy Sauce


In [5]:
def standardize(input):
    lwer = tf.strings.lower(input)
    punc = tf.strings.regex_replace(lwer, '[%s]' % re.escape(string.punctuation), '')
    return punc 

In [6]:
max_tokens = 5000
sequence_length = 20

vectorize = TextVectorization(
    standardize = standardize,
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = sequence_length) 

In [44]:
data = tf.data.Dataset.from_tensor_slices(df) # convert to TensorFlow Dataset

In [45]:
vectorize.adapt(data) # set up the vectorizing

In [46]:
for recipe in data.take(5):
    print(recipe)

tf.Tensor([b'Salmon & Salad a La SPORTZ'], shape=(1,), dtype=string)
tf.Tensor([b'Curried Pumpkin and Smoked Salmon Soup'], shape=(1,), dtype=string)
tf.Tensor([b'Grilled Rosemary Salmon Spedini'], shape=(1,), dtype=string)
tf.Tensor([b'Garlic and Dill Salmon'], shape=(1,), dtype=string)
tf.Tensor([b'Spicy Grilled Orange Salmon'], shape=(1,), dtype=string)


In [53]:
def vectorize_recipe(text):
    text = tf.expand_dims(text, -1)
    return vectorize(text)

data_vec = data.map(vectorize_recipe) # convert recipes to numbers

In [52]:
list(data_vec.take(5))

[<tf.Tensor: shape=(1, 20), dtype=int64, numpy=
 array([[  2,   8,  27, 260, 899,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0]], dtype=int64)>,
 <tf.Tensor: shape=(1, 20), dtype=int64, numpy=
 array([[158, 683,   4,   5,   2, 100,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0]], dtype=int64)>,
 <tf.Tensor: shape=(1, 20), dtype=int64, numpy=
 array([[   7,  140,    2, 2221,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]],
       dtype=int64)>,
 <tf.Tensor: shape=(1, 20), dtype=int64, numpy=
 array([[49,  4, 11,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]], dtype=int64)>,
 <tf.Tensor: shape=(1, 20), dtype=int64, numpy=
 array([[40,  7, 52,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]], dtype=int64)>]

In [12]:
vocabulary = vectorize.get_vocabulary() # collect all the words used

In [13]:
def num_to_str(vec, vocab):
    ''' converts numeric recipe to original title '''
    arr = vec.numpy()
    arr = arr[0]
    title = ""
    for num in arr:
        title += vocab[num] + " "
    return title

In [14]:
for item in data_vec:
    out = num_to_str(item, vocabulary)
    print(out)

salmon salad a la sportz                
curried pumpkin and smoked salmon soup               
grilled rosemary salmon spedini                 
garlic and dill salmon                 
spicy grilled orange salmon                 
roast salmon with spiced coconut crumbs               
baked salmon with tomatoes spinach mushrooms               
grilled salmon northwest style                 
smoked salmon chive cream cheese crispy mashed potato waffles            
salmon with lemon sauce and linguine               
salmon loaf                   
broiled salmon with spicy maple basting sauce              
whipped chive and smoked salmon bagel               
double salmon cakes                  
uncle bills salmon marinated in maple syrup and soy sauce           
scrambled eggs with dill and smoked salmon              
grilled salmon with mustard sauce                
sockeye salmon crostini                  
baked marinated salmon                  
cedar salmon with wasabi dip             

eggceptional smoked salmon scramble                 
rosemary roasted salmon                  
healthy salmon cakes                  
salmon penne                   
salmon with hoisin orange and bok choy              
pan roasted salmon with fava beans and roasted tomatoes            
smoked salmon and pea frittata recipe               
easy asparagus salmon egg scramble for one              
brown bread with smoked salmon                
smoked salmon tartare                  
baked lemon salmon with lemondill sauce               
grilled salted salmon                  
moist poached salmon                  
fillet of salmon in a porcini pumpkin seed crust            
salmon jerky candy recipe                 
fresh salmon nuggets                  
cedarplanked salmon with washington state merlot reduction and garlic spinach           
salmon platter with caper dressing                
salmon and broccoli bake                 
salmon pasta recovery meal                 
smoked salmon

stirfried vegetables with salmon                 
salmon puffs                   
dill salmon dip                  
salted salmon and tomato salad with onions aka lomi lomi salmon          
roasted garlic potato soup with smoked salmon              
dilly salmon tostada with wasabi mustard               
baked salmon ii                  
misoglazed salmon with wilted spinach                
salmon pecancrusted tartlets                  
pama barbecued salmon                  
bourbon glazed salmon recipe                 
salmon with yogurtcurry sauce                 
smoked salmon and creme fraiche canape               
dilled salmon salad recipe                 
salmon pattiesburgers with a yogurt herb sauce              
asian salmon                   
smoked salmon pizza a la wolfgang               
poached salmon for two                 
golden salmon on a bed of lentils              
simply grilled wild sockeye salmon                
irish roasted salmon                  
rich but

salmon in butter sauce top crispy garlic              
misocured salmon with asparagus and black garlic sauce             
warm salmon dip 2 ingredients                
warm summer salad with salmon                
the ultimate salmon in parchment                
salmon pie i                  
salmon with pickled carrots and baconfried potatoes              
fusion roasted salmon                  
quick salmonpotato cakes                  
yankee salmon chowder                  
baked salmon recipe                  
bourbon grilled salmon                  
beas smoked salmon baguettes                 
simple and healthy poached salmon                
honey teriyaki salmon                  
smoked salmon cream cheese souffle                
absolut vodka cured salmon with blood orange juice and li recipe          
panbroiled teriyaki salmon lowfat                 
delicate crepe and smoked salmon gateau               
gingersalmon cake sandwiches                  
maple barbecue salmon 

black pepper yogurt salmon                 
salmon chowder                   
penne with smoked salmon and peas               
panseared salmon with mediterranean salad                
panseared salmon with curried vegetables                
quick cured salmon                  
salmon souvlaki                   
grilled salmon with citrus butter                
open faced salmon sandwich                 
saumon a lunilateral salmon cooked on one side only            
herbed citrus salmon                  
baked salmon with creole mustard sauce               
wolfgang pucks salmon with celeryroot puree               
baked salmon in parchment with julienned vegetables              
salmon fillet                   
salmon salad a la jacque                
poached salmon with coriander sauce                
salmon with mango salsa                 
baked salmon                   
smoked salmon squaquerone cheese red onion and arugula             
mughlai masala salmon                  
poa

...

grilled salmon with greekstyle vegetables                
grilled salmon steaks with lime butter               
herbroasted salmon and tomatoes                 
smoked salmon and blue cheese cheesecake with an avocado cream topping          
salmon with asparagus sauce                 
dads candied salmon                  
chinese glazed salmon                  
smoked salmon reuben                  
baked salmon in blue cheese creamy paradise              
pestocrusted salmon fillet with citrussoy sauce               
salmon with pearl onions lettuce and peas              
salmon mousse on dill pizzelle                
salmon burgers with horseradishdill sauce                
pecan coated salmon fillet                 
johnny garlics cedar plank salmon                
highspeed salmon grill                  
seared salmon with linguine and ramp pesto              
fiesta baked salmon                  
misoginger glazed salmon recipe                 
soba with smoked salmon cucumberand

easy salmon quiche                  
mini fresh salmon cakes with cucumber aioli              
panfried salmon fillets with mangoavocado salsa               
smoked salmon and capers in a champagne sauce for pasta           
salmon cakes with gingersesame sauce                
salmon with white beans                 
chardonnay poached salmon with fennel and orange              
grilled side of salmon                 
poached salmon with zucchini recipe                
smoked salmon panini                  
baked salmon with herb sauce recipe               
smoked salmon and cream cheese omelet               
salmon with puff pastry and pesto               
grilled honeybalsamic salmon                  
15 minute salmon bisque                 
red salmon mousse                  
grilled salmon with dill butter                
salmon and potato salad with lemon poppy seed dressing            
curry salmon                   
spicy garlic salmon                  
teriyaki salmon          

firecracker salmon oamc                  
salmon and cucumber terrine                 
potato soup with smoked salmon relish               
salmon and potato piccata                 
chili salmon indian style                 
salmon pasta salad                  
grilled lime garlic scallion smoked paprika salmon              
pecancrusted salmon                   
smoked salmon and basil rolls with creme fraiche or sour cream          
salmon egg on toast                 
yummy lemon salmon burgers                 
pickled salmon                   
salmon burgers                   
lemonbuttered salmon                   
salmon and pea tagliatelle                 
salmon macaroni salad                  
smoked salmon on sourdough rounds with caperdill relish             
soy poached salmon                  
crisp and crunchy salmon or chicken fingers              
polynesian glazed salmon                  
baked salmon with creamy dill sauce recipe              
salmon cakes recipe    

salmon with white wine sauce and fettuccini              
salmon in parchment with green beans and lemon zest            
salmon on greens with lime ginger dressing              
grilled salmon with coffee barbecue sauce               
smoked salmon and leek tartlets                
alaskan salmon marinade                  
salmon macaroni scallop                  
salmon cake markii                  
salmon and caviar pie                 
blue cheese salmon meatballs with sriracha dipping sauce             
salmon with chive oil cauliflower and greens              
salmon wellington                   
poached salmon with sauteed leeks and leekcream sauce             
panroasted salmon with orange vinaigrette                
salmon stuffed in puff pastry                
smoked salmon and onion tart                
whole roasted sake salmon in banana leaves              
cedarplanked wild alaska salmon with mustarddill sauce              
norwegian smoked salmon roll with cream cheese  

salmon gravy                   
salmon sandwiches                   
poached salmon with lime and ginger sauce              
salmon with tomato vinaigrette                 
salmon grilled                   
soy maple salmon                  
salmon noodle loaf recipe                 
salmon cakes                   
smoked salmon and baby potato salad wedding buffet             
brown sugarcured salmon                  
salmon pink tamagoyaki made with leftover sakura denbu             
the best salmon pasta bake                
barbecued salmon sandwiches                  
salmon frittata slimming world friendly                
roast salmon with beetroot kale and pomegranate salad recipe            
easy salmon chowder                  
salmon and asparagus chinois                 
quick smoked salmon dip                 
coriandercrusted salmon                   
indianstyle salmon fry                  
salmon pate                   
salmon tournedos with herb sauce                
es

In [23]:
def split_input_target(title):
    input_text = title[:-1]
    target_text = title[1:]
    
    return input_text, target_text

dataset_targeted = data_vec.map(split_input_target)

print(dataset_targeted)

<MapDataset shapes: ((None, 20), (None, 20)), types: (tf.int64, tf.int64)>


In [38]:
list(dataset_targeted.take(1).take(1))

[(<tf.Tensor: shape=(0, 20), dtype=int64, numpy=array([], shape=(0, 20), dtype=int64)>,
  <tf.Tensor: shape=(0, 20), dtype=int64, numpy=array([], shape=(0, 20), dtype=int64)>)]

In [39]:
list(data_vec.take(1))

[<tf.Tensor: shape=(1, 20), dtype=int64, numpy=
 array([[  2,   8,  27, 260, 899,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0]], dtype=int64)>]

In [16]:
VOCABULARY_SIZE = max_tokens +1

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        batch_input_shape=[batch_size, None]
    ))

    model.add(tf.keras.layers.LSTM(
        units=rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer=tf.keras.initializers.GlorotNormal()
    ))

    model.add(tf.keras.layers.Dense(vocab_size))
    
    return model

model = build_model(
  vocab_size=VOCABULARY_SIZE,
  embedding_dim=256,
  rnn_units=1024,
  batch_size= sequence_length
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (20, None, 256)           1280256   
_________________________________________________________________
lstm (LSTM)                  (20, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (20, None, 5001)          5126025   
Total params: 11,653,257
Trainable params: 11,653,257
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam', 
              metrics=['accuracy'])

In [32]:
history = model.fit(dataset_targeted, epochs = 10)

Epoch 1/10


InvalidArgumentError:  Input to reshape is a tensor with 0 values, but the requested shape has 400
	 [[node gradient_tape/sequential/embedding/embedding_lookup/Reshape_1 (defined at <ipython-input-32-5d6ff96fa266>:1) ]] [Op:__inference_train_function_46174]

Function call stack:
train_function
